In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy.linalg as la
import pandas as pd
from scipy.linalg import svd
from mpl_toolkits.mplot3d import Axes3D
import random
from pipeline import generate_basic_exploration

In [74]:
df = pd.read_csv('allegations_cleaned2.csv')

In [76]:
df['rank_abbrev_incident'].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: rank_abbrev_incident, dtype: float64

In [77]:
missing_officer_vars = df[['officer_id', 'mos_gender', 'mos_age_incident', 
                           'rank_abbrev_incident']]

In [79]:
mov = missing_officer_vars.groupby('officer_id').mean()

In [85]:
complaint_counts = missing_officer_vars.groupby('officer_id').count()

In [86]:
complaint_counts 

,mos_gender,mos_age_incident,rank_abbrev_incident
officer_id,,,
2,10,10,10
5,3,3,3
11,5,5,5
21,8,8,8
23,14,14,14
...,...,...,...
36301,3,3,3
36305,3,3,3
36317,2,2,2


In [87]:
mov['complaint_count'] = complaint_counts['mos_gender']

In [88]:
mov.head()

,mos_gender,mos_age_incident,rank_abbrev_incident,complaint_count
officer_id,,,,
2,0.0,33.700000,2.000000,10
5,1.0,33.666667,1.666667,3
11,0.0,27.800000,1.000000,5
21,0.0,29.750000,1.000000,8
23,0.0,24.857143,1.000000,14


In [75]:
list(df.columns)

['Unnamed: 0',
 'Unnamed: 0.1',
 'rank_abbrev_incident',
 'mos_gender',
 'mos_age_incident',
 'same_ethnicity',
 'closure_time',
 'demotion',
 'promotion',
 'rank_abbrev_now',
 'dispute',
 'vehicle',
 'suspected_violation',
 'phone',
 'warrant',
 'aided_case',
 'checkpoint',
 'demonstrations_protests',
 'report',
 'disturbance',
 'precinct_complaint_info',
 'vehicle_violation',
 'arrest_resist',
 'arrest_obstruct',
 'other_summons',
 'Arrest/Complainant',
 'C/V intervened on behalf of/observed encounter w/3rd party',
 'C/V at PCT to file complaint of crime',
 'Regulatory inspection',
 'Assist ACS or other agency',
 'CV already in custody',
 'Report-gun possession/shots fired',
 'Stop/Question/Frisk',
 'C/V requested info from officer',
 'Patrol Encounter',
 'Arrest/Not Complainant',
 'Summons/Complainant',
 'Complainant Witnessing Incident',
 'C/V at PCT to retrieve property',
 'Parade/special event',
 'Victim Subject of Sex Crime',
 'No contact',
 'Arrest - other violation/crime',
 'N

In [37]:
df['board_disposition'].head(10)

0    Substantiated (Command Lvl Instructions)
1                     Substantiated (Charges)
2                     Substantiated (Charges)
3                     Substantiated (Charges)
4        Substantiated (Command Discipline A)
5    Substantiated (Command Lvl Instructions)
6         Substantiated (Formalized Training)
7                             Unsubstantiated
8                             Unsubstantiated
9        Substantiated (Command Discipline A)
Name: board_disposition, dtype: object

In [38]:
df['substantiated'] = [1 if x not in ('Exonerated', 'Unsubstantiated')
                       else 0 for x in df['board_disposition']]

In [42]:
df['officer_charged'] = df['officer_charged'].astype(int)

In [5]:
demo_vars = ['c_black',
 'c_unknown',
 'c_white',
 'c_asian',
 'c_hispanic',
 'c_american_indian',
 'c_female',
 'c_male',
 'c_transman_(ftm)',
 'c_transwoman_(mtf)',
 'c_gender_non-conforming']

In [11]:
officer_demos = df.groupby(['officer_id', 'complaint_id']).mean()

In [15]:
officer_demos = officer_demos[demo_vars]

In [18]:
for col in officer_demos.columns:
    officer_demos[col] = [np.sign(x) for x in officer_demos[col]]

In [21]:
officer_demo_sums = officer_demos.groupby('officer_id').sum()

In [25]:
officer_demo_sums = officer_demo_sums.reset_index()

In [69]:
officer_demo_sums.head()

,officer_id,c_black,c_unknown,c_white,c_asian,c_hispanic,c_american_indian,c_female,c_male,c_transman_(ftm),c_transwoman_(mtf),c_gender_non-conforming
0,2,6.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0
1,5,0.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
2,11,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
3,21,0.0,0.0,3.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
4,23,6.0,1.0,0.0,0.0,1.0,0.0,2.0,6.0,0.0,0.0,0.0


In [60]:
exclude_cols =  ['Unnamed: 0',
                 'Unnamed: 0.1',
                 'rank_abbrev_incident',
                 'mos_gender',
                 'mos_age_incident',
                 'same_ethnicity',
                 'closure_time',
                 'rank_abbrev_now',
                 'outcome',
                 'complaint_id',
                 'board_disposition',
                 'mos_hispanic',
                 'mos_black',
                 'mos_asian',
                 'mos_american_indian'] + demo_vars

In [61]:
officer_vars = df.drop(columns=exclude_cols)

In [62]:
officer_var_sums = officer_vars.groupby('officer_id').sum()

In [63]:
officer_var_sums['ever_charged'] = np.sign(officer_var_sums['officer_charged'])
officer_var_sums['demotion'] = np.sign(officer_var_sums['demotion'])
officer_var_sums['promotion'] = np.sign(officer_var_sums['promotion'])

In [64]:
sum(officer_var_sums['ever_charged'])/len(officer_var_sums)

0.43743743743743746

In [67]:
sum(officer_var_sums['demotion'])/len(officer_var_sums)

0.017767767767767766

In [68]:
officer_var_sums = officer_var_sums.reset_index()

In [70]:
officer_df = pd.merge(officer_demo_sums, officer_var_sums, on='officer_id')

In [72]:
len(officer_df)

3996

In [91]:
officer_df.head(20)

,Unnamed: 0,officer_id,c_black,c_unknown,c_white,c_asian,c_hispanic,c_american_indian,c_female,c_male,...,Profane Gesture,Animal,Gender Identity,officer_charged,substantiated,ever_charged,mos_gender,mos_age_incident,rank_abbrev_incident,complaint_count
0,0,2,6.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,...,0.0,0.0,0.0,1,1,1,0.0,33.700000,2.000000,10
1,1,5,0.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0,1,0,1.0,33.666667,1.666667,3
2,2,11,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,2,2,1,0.0,27.800000,1.000000,5
3,3,21,0.0,0.0,3.0,1.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,4,4,1,0.0,29.750000,1.000000,8
4,4,23,6.0,1.0,0.0,0.0,1.0,0.0,2.0,6.0,...,0.0,0.0,0.0,1,1,1,0.0,24.857143,1.000000,14
5,5,28,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0,3,0,0.0,29.400000,1.000000,5
6,6,38,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,1,1,0.0,27.000000,1.000000,3
7,7,43,2.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,1,1,1,1.0,26.800000,1.000000,5
8,8,73,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0,2,0,0.0,25.000000,1.000000,2
9,9,83,2.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,2,3,1,0.0,32.250000,1.000000,8


In [89]:
officer_df = pd.read_csv('officer_df.csv')

In [90]:
officer_df = pd.merge(officer_df, mov.reset_index(), on='officer_id')

In [92]:
officer_df.to_csv('officer_df.csv')